In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import pickle
import os

In [2]:
from load_data import *
from generate_set import *
from xgb_predict import *
from train_xgb import *

In [3]:
from sklearn.metrics import mean_squared_error as mse
from math import sqrt

In [4]:
t_raw = pd.read_pickle(os.path.join('dataframes', 'train_raw.pkl'))
v_raw = pd.read_pickle(os.path.join('dataframes', 'validation_raw.pkl'))
test_b1 = load_raw_file(os.path.join('data', 'ai_challenger_wf2018_testb7_20180829-20181103.nc'))
all_raw = pd.concat([t_raw, v_raw, test_b1])

Loading netCDF4 file data/ai_challenger_wf2018_testb7_20180829-20181103.nc
Loading station 90001
Loading station 90002
Loading station 90003
Loading station 90004
Loading station 90005
Loading station 90006
Loading station 90007
Loading station 90008
Loading station 90009
Loading station 90010


In [5]:
with open(os.path.join('dataframes', 'test_b6.pkl'), 'wb') as file:
    pickle.dump(test_b1, file)

In [6]:
# TODO: Change this prediction date every day!
predict_datetime = '2018-11-03 03:00:00'

In [17]:
x_oneday, y_oneday = generate_x_oneday(predict_datetime, all_raw)

# With early stopping: 'models', '1015_1115_2', '2'
# Without early stopping: 'models', '1015_1115_2', '3'
# Modified validation set: 'models', '1015_1115_2', '4'
pre_xgb = xgb_predict_all(x_oneday, y_oneday, os.path.join('models', '1015_1115_2', '4'))

/Users/loganlin/Downloads/ai-challenger/generate_set.py:69: PerformanceWarning: indexing past lexsort depth may impact performance.
  history_list.append(df.loc[station_id, forecast_date - pd.DateOffset(days=i)].iloc[:24])
/Users/loganlin/Downloads/ai-challenger/generate_set.py:70: PerformanceWarning: indexing past lexsort depth may impact performance.
  history = pd.concat(history_list + [df.loc[station_id, forecast_date]])
/Users/loganlin/Downloads/ai-challenger/generate_set.py:154: PerformanceWarning: indexing past lexsort depth may impact performance.
  history_list.append(df.loc[station_id, forecast_date - pd.DateOffset(days=i)].iloc[:24])
/Users/loganlin/Downloads/ai-challenger/generate_set.py:157: PerformanceWarning: indexing past lexsort depth may impact performance.
  h = normalize(pd.concat(history_list + [df.loc[station_id, forecast_date]]))


In [19]:
dl_pre = load_numpy_prediction(os.path.join('numpy', 'pred_result_11_03.npy'))
merge_pre = (dl_pre.sort_index() + pre_xgb.sort_index()) / 2

In [20]:
sub = construct_submission(all_raw, dl_pre, predict_datetime)

/Users/loganlin/anaconda3/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  


In [7]:
def load_numpy_prediction(file_path):
    """
    Turn Deep Learning output numpy array into the same format as XGBoost prediction result.
    """
    pre_np = np.load(file_path)
    pre_np = pre_np.reshape([3, 330]).T
    indexes = pd.concat([pd.Series(['%d_%02d' % (station, seq) for seq in range(4, 37)]) for station in range(90001, 90011)])
    indexes.name = 'FORE_data'
    pre_df = pd.DataFrame(pre_np, columns=['t2m_obs', 'w10m_obs', 'rh2m_obs'], index=indexes)
    pre_df = pre_df[['t2m_obs', 'rh2m_obs', 'w10m_obs']]
    return pre_df

In [8]:
def merge_prediction(xgb_pre: pd.DataFrame, dl_pre: pd.DataFrame, merge_rule:pd.DataFrame, linear=False) -> pd.DataFrame:
    """
    Using merge_rule to merge XGBoost prediction and Deep Learning prediction.
    """
    xgb_pre = xgb_pre.sort_index()
    dl_pre = dl_pre.sort_index()
    merge_rule = merge_rule.sort_index()
    
    result = []
    
    station_list = list(range(90001, 90011))
    for column in xgb_pre.columns:
        result_column = []
        for i in range(0, 10):
            station = station_list[i]
            rmse_diff = merge_rule.loc[station, column]
            if rmse_diff > 0.5:
                if linear:
                    result_column.append((dl_pre.iloc[(i * 33):((i + 1) * 33)][column] + 
                                         xgb_pre.iloc[(i * 33):((i + 1) * 33)][column]) / 2)
                else:
                    result_column.append(dl_pre.iloc[(i * 33):((i + 1) * 33)][column])
            else:
                result_column.append(xgb_pre.iloc[(i * 33):((i + 1) * 33)][column])
        result.append(pd.concat(result_column))
    result = pd.DataFrame(result).T
    
    return result

In [9]:
def add_cycle_to_X(X, raw, column, predict_hour, add_days, interval=1):
    explain = pd.read_csv(os.path.join('data', 'explain.csv'), index_col=0).dropna(how='any', axis=1)
    
    def get_station_id(X):
        station_list = []
        for index, row in X.iterrows():
            station_onehot = row[['station_{}'.format(i) for i in range(10)]]
            station_id = int('900%02d' % int(np.sum([station_onehot[i] * (i+1) for i in range(10)])))
            station_list.append(station_id)
        return station_list

    def normalize_single_column(df: pd.DataFrame, name) -> pd.DataFrame:
        """
        Normalize data.
        """
        result = pd.DataFrame(df, copy=True)
        scope = explain.loc[name]['scope'][1:-1].split(',')
        min_value = float(scope[0].strip())
        max_value = float(scope[1].strip())

        result[name] = (result[name] - min_value) / (max_value - min_value)
        return result
    
    origin_dates = [pd.Timestamp.fromtimestamp(timestamp) - pd.Timedelta('8 hours') for timestamp in X['timestamp']]
    station_list = get_station_id(X)
    series_list = []
    for day in range(add_days):
        day = (day + 1) * interval
        fetch_dates = pd.Series(origin_dates) - pd.Timedelta('{} days'.format(day))
        series = pd.Series([raw.loc[(station_id, date, predict_hour + 4)][column] 
                            for (station_id, date) in zip(station_list, fetch_dates)])
        series.name = column
        series_list.append(series)
    result = normalize_single_column(pd.DataFrame(series_list).T, column)
    result.columns = ['{}_{}*{}days'.format(column, interval, i + 1) for i in range(add_days)]
    return pd.concat([X, result], axis=1)

In [10]:
def xgb_predict_all(X, Y, dir_prefix):
    """
    Predict all timestamps and features using xgb models.
    """
    result_list = []
    for i in range(Y[0].shape[0]):
        result_one_timestamp = []
        for j in range(Y[0].shape[1]):
            name = Y[0].columns[j]
            
            # X_added
            X_added = add_cycle_to_X(X, all_raw, name, i, 7)
#             X_added = add_cycle_to_X(X_added, all_raw, name, i, 4, 7)
            
            xg_reg = pickle.load(open('{}_{}_{}.model'.format(dir_prefix, name, i), 'rb'))
            
            # X_added
            result = pd.Series(xg_reg.predict(X_added))
    
            indexes = ['%d_%02d' % (station_id, i + 4) for station_id in range(90001, 90011)]
            result.index = indexes
            result_one_timestamp.append(result)
        result_list.append(pd.DataFrame(result_one_timestamp).T)
    result = pd.concat(result_list)
    result.columns = Y[0].columns
    result.index.name = 'FORE_data'
    result = denormalize(result)
    return result

In [11]:
def eval_timespan(start, end, eval_set, dir_prefix, days=2):
    """
    Use evaluation set to generate a summarize of 
    """
    start = pd.Timestamp(start)
    end = pd.Timestamp(end)
    y_list = []
    pre_list = []
    for date in pd.date_range(start, end):
        x, y = generate_x_oneday(date, eval_set, days)
        y_list.append(y)
        pre = xgb_predict_all(x, y, dir_prefix)
        pre_list.append(pre)
    
    # Calculate RMSE for each hour
    hour_list = []
    columns = None
    for i in range(33):
        pre_hour = pd.concat([pre.iloc[(i*10):((i+1)*10)] for pre in pre_list])
        y_hour = denormalize(pd.concat([pd.DataFrame([y_one_station.iloc[i] for y_one_station in y]) for y in y_list]))
        columns = y_hour.columns
        row = []
        for column in columns:
            row.append(mse(pre_hour[column], y_hour[column]))
        hour_list.append(row)
    hour_rmse = pd.DataFrame(np.sqrt(hour_list), columns=columns)
    
    # Calculate RMSE for each station
    station_list = []
    for i in range(10):
        pre_station = pd.concat([pre.sort_index().iloc[(i*33):((i+1)*33)] for pre in pre_list])
        y_station = denormalize(pd.concat([y[i] for y in y_list]))
        row = []
        for column in columns:
            row.append(mse(pre_station[column], y_station[column]))
        station_list.append(row)
    station_rmse = pd.DataFrame(np.sqrt(station_list), columns=columns, index=range(90001, 90011))
    
    # Calculate overall RMSE for three features
    all_rmse = pd.DataFrame(np.sqrt(np.mean(station_list, axis=0)), index=columns)
    
    return hour_rmse, station_rmse, all_rmse

In [105]:
hour_rmse, station_rmse, all_rmse = eval_timespan('2018-10-25 03:00:00', '2018-10-26 03:00:00', 
                                                  all_raw, os.path.join('models', '1015_1115_2', '2'))

/data/LinYan/ai-challenger/generate_set.py:69: PerformanceWarning: indexing past lexsort depth may impact performance.
  history_list.append(df.loc[station_id, forecast_date - pd.DateOffset(days=i)].iloc[:24])
/data/LinYan/ai-challenger/generate_set.py:70: PerformanceWarning: indexing past lexsort depth may impact performance.
  history = pd.concat(history_list + [df.loc[station_id, forecast_date]])
/data/LinYan/ai-challenger/generate_set.py:154: PerformanceWarning: indexing past lexsort depth may impact performance.
  history_list.append(df.loc[station_id, forecast_date - pd.DateOffset(days=i)].iloc[:24])
/data/LinYan/ai-challenger/generate_set.py:157: PerformanceWarning: indexing past lexsort depth may impact performance.
  h = normalize(pd.concat(history_list + [df.loc[station_id, forecast_date]]))


In [107]:
# dl_rmse = [[ 1.8833348,3.49686628,3.8517941 ,  3.47419742 , 1.25979552 , 4.76592387,
#    3.83747535 , 1.94121611 , 3.57586791  ,3.21245561],
#  [ 1.39719772,  1.24978646  ,1.2362636  , 1.33176675 , 1.45978062 , 1.27638727,
#    1.87945335 , 1.68263699 , 1.08730759 , 1.64784673],
#  [15.07691953, 11.74088683, 16.04245051, 14.7884658 , 13.68954934, 23.22244553,
#    8.15870517 , 4.72383667,  5.80455138 ,15.42624515]]
# dl_rmse = np.array(dl_rmse).T
dl_rmse = np.load(os.path.join('numpy', 'final_rmse.npy')).T
dl_rmse = pd.DataFrame(dl_rmse, index=range(90001, 90011), columns=['t2m_obs' ,'w10m_obs' ,'rh2m_obs'])
dl_rmse = dl_rmse[['t2m_obs', 'rh2m_obs', 'w10m_obs']]

In [12]:
def plot_prediction(y, prediction):
    rmse_matrix = []
    for i in range(len(y)):
        rmse_one = []
        plt.figure(figsize=(20, 3))
        for j in range(3):
            column = prediction.columns[j]
            plt.subplot(1, 3, j+1, ylabel=column)
            if j == 1:
                plt.title(range(90001, 90011)[i])
            pre_one = prediction.sort_index().iloc[33*i:33*(i+1)][column]
            y_one = denormalize(y[i])[column]
            plt.plot(range(33), pre_one, color='red', label='pre')
            plt.plot(range(33), y_one, color='green', label='true')
            rmse_one.append(sqrt(mse(pre_one, y_one)))
            plt.legend()
        rmse_matrix.append(rmse_one)
    return pd.DataFrame(rmse_matrix, columns=prediction.columns, index=range(90001, 90011))

In [149]:
dl_pre = load_numpy_prediction(os.path.join('numpy', 'pred_result_10_28.npy'))

In [112]:
x_1015, y_1015 = generate_x_oneday('2018-10-15 03:00:00', all_raw)

/data/LinYan/ai-challenger/generate_set.py:69: PerformanceWarning: indexing past lexsort depth may impact performance.
  history_list.append(df.loc[station_id, forecast_date - pd.DateOffset(days=i)].iloc[:24])
/data/LinYan/ai-challenger/generate_set.py:70: PerformanceWarning: indexing past lexsort depth may impact performance.
  history = pd.concat(history_list + [df.loc[station_id, forecast_date]])
/data/LinYan/ai-challenger/generate_set.py:154: PerformanceWarning: indexing past lexsort depth may impact performance.
  history_list.append(df.loc[station_id, forecast_date - pd.DateOffset(days=i)].iloc[:24])
/data/LinYan/ai-challenger/generate_set.py:157: PerformanceWarning: indexing past lexsort depth may impact performance.
  h = normalize(pd.concat(history_list + [df.loc[station_id, forecast_date]]))


In [13]:
def construct_submission(raw_data, prediction, forecast_date, file_dir=None):
    """
    Construct submission data frame using original data and forecast result.
    """
    forecast_date = pd.Timestamp(forecast_date)
    origin_list = []
    for station_id in range(90001, 90011):
        origin_data = raw_data.loc[station_id, forecast_date][prediction.columns].iloc[:4]
        origin_data = origin_data.interpolate(limit_direction='both')
        origin_data.index = ['%d_%02d' % (station_id, i) for i in range(4)]
        origin_list.append(origin_data)
    origin = pd.concat(origin_list)
    submission = pd.concat([origin, prediction])
    submission.columns = [name.split('_')[0] for name in submission.columns]
    submission.index.name = 'FORE_data'
    submission = submission.sort_index()

    if file_dir is None:
        file_dir = os.path.join('submission', 'forecast-{}.csv'.format(forecast_date.strftime('%Y%m%d%H')))
    with open(file_dir, 'w') as f:
        f.write('%10s,%10s,%10s,%10s' % ('FORE_data', 't2m', 'rh2m', 'w10m'))
        for index, value in submission.iterrows():
            f.write('\n%10s,%10.5f,%10.5f,%10.5f' % (index, value['t2m'], value['rh2m'], value['w10m']))
    return submission